# Basis _reinforcement learning_

## CartPole

CartPole is een klassiek controle probleem waarbij een staaf rechtop moet blijven op een kar die heen en weer kan bewegen.

[![CartPole](https://gymnasium.farama.org/_images/cart_pole.gif)](https://gymnasium.farama.org/)

### Het Probleem
- **State**: 4 continue waarden (positie kar, snelheid kar, hoek staaf, hoeksnelheid staaf)
- **Actions**: 2 discrete acties (duw naar links of rechts)
- **Reward**: +1 voor elke tijdstap waarbij de staaf rechtop blijft
- **Doel**: Hou de staaf zo lang mogelijk rechtop (max 500 tijdstappen)

### Setup
We gebruiken:
- [Gymnasium](https://gymnasium.farama.org/index.html): Een framework voor RL omgevingen (oorspronkelijk van OpenAI)
- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/#): Kwalitatieve PyTorch implementaties van RL algoritmes

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd
import plotly.express as px
import torch
# from matplotlib import animation


# print(f"CUDA available: {torch.cuda.is_available()}")

### Stap 1: De Omgeving Verkennen

In [2]:
# Create the CartPole environment
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Reset environment to get initial state
state, info = env.reset(seed=42)

print("=== CartPole Environment ===")
print(f"State space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"\nInitial state: {state}")
print("\nState components:")
print(f"  [0] Cart Position: {state[0]:.3f}")
print(f"  [1] Cart Velocity: {state[1]:.3f}")
print(f"  [2] Pole Angle: {state[2]:.3f}")
print(f"  [3] Pole Angular Velocity: {state[3]:.3f}")
print("\nPossible actions:")
print("  0: Push cart to the LEFT")
print("  1: Push cart to the RIGHT")

=== CartPole Environment ===
State space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Action space: Discrete(2)

Initial state: [ 0.0273956  -0.00611216  0.03585979  0.0197368 ]

State components:
  [0] Cart Position: 0.027
  [1] Cart Velocity: -0.006
  [2] Pole Angle: 0.036
  [3] Pole Angular Velocity: 0.020

Possible actions:
  0: Push cart to the LEFT
  1: Push cart to the RIGHT


### Stap 2: Baseline agent

Voordat we een intelligent model trainen, kijken we eerst hoe een **random agent** (die willekeurige acties neemt) presteert. Dit geeft ons een baseline.

In [3]:
# Test random agent
def evaluate_random_agent(env, n_episodes=10, seed=42):
    """
    Evaluate a random agent that takes random actions.

    Args:
        env: Gymnasium environment
        n_episodes: Number of episodes to run
        seed: Random seed for reproducibility

    Returns
    -------
        List of episode rewards
    """
    episode_rewards = []

    for episode in range(n_episodes):
        state, info = env.reset(seed=seed + episode)
        episode_reward = 0
        done = False
        truncated = False

        while not (done or truncated):
            # Random action
            action = env.action_space.sample()
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward

        episode_rewards.append(episode_reward)

    return episode_rewards


# Evaluate random agent
random_rewards = evaluate_random_agent(env, n_episodes=100)

print("=== Random Agent Performance ===")
print(f"Average reward: {np.mean(random_rewards):.2f} ± {np.std(random_rewards):.2f}")
print(f"Min reward: {np.min(random_rewards):.2f}")
print(f"Max reward: {np.max(random_rewards):.2f}")

px.histogram(random_rewards, nbins=20, title="Random Agent: Reward Distribution").add_vline(
    x=np.mean(random_rewards),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean: {np.mean(random_rewards):.1f}",
).show()

=== Random Agent Performance ===
Average reward: 21.69 ± 10.88
Min reward: 9.00
Max reward: 65.00


In [4]:
px.line(
    y=random_rewards,
    title="Random Agent: Reward per Episode",
    labels={"x": "Episode", "y": "Reward"},
).add_hline(
    y=np.mean(random_rewards),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean: {np.mean(random_rewards):.1f}",
).show()


### Stap 3: Training met Deep Q-Network (DQN)

Nu gaan we een **Deep Q-Network (DQN)** trainen om een intelligente policy te leren. DQN is een value-based methode die een neural network gebruikt om de optimale $Q$-functie $Q^*(s,a)$ te benaderen.

In [5]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [6]:
# Create a fresh environment for training
env = gym.make("CartPole-v1")

# Create DQN model with better hyperparameters
# The neural network will learn Q(s,a) for each state-action pair
model = DQN(
    "MlpPolicy",  # Multi-Layer Perceptron policy network
    env,
    learning_rate=1e-3,
    buffer_size=50000,
    learning_starts=1000,  # Start learning after more experiences
    batch_size=64,  # Larger batch size for more stable learning
    tau=1.0,
    gamma=0.99,  # Discount factor
    train_freq=4,
    target_update_interval=250,
    exploration_fraction=0.1,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.02,  # Lower final exploration
    verbose=1,  # Show training progress
    tensorboard_log=None,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
# Train the agent for longer
model.learn(total_timesteps=100000, progress_bar=True)

# Evaluate the trained model (wrap env with Monitor to avoid warning)
eval_env = Monitor(gym.make("CartPole-v1"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100, deterministic=True)
eval_env.close()

print("\n=== Trained DQN Agent Performance ===")
print(f"Mean reward: {mean_reward:.2f} ± {std_reward:.2f}")
print("\nComparison:")
print(f"  Random Agent: {np.mean(random_rewards):.2f} ± {np.std(random_rewards):.2f}")
print(f"  Trained DQN:  {mean_reward:.2f} ± {std_reward:.2f}")
if mean_reward > np.mean(random_rewards):
    print(f"  Improvement:  {((mean_reward / np.mean(random_rewards)) - 1) * 100:.1f}%")
else:
    print(f"  Performance:  {(mean_reward / np.mean(random_rewards)) * 100:.1f}% of random agent")
    print("  ⚠️  Model needs more training or hyperparameter tuning!")

Output()

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 31       |
|    ep_rew_mean      | 31       |
|    exploration_rate | 0.988    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1531     |
|    time_elapsed     | 0        |
|    total_timesteps  | 124      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 26.1     |
|    ep_rew_mean      | 26.1     |
|    exploration_rate | 0.98     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2191     |
|    time_elapsed     | 0        |
|    total_timesteps  | 209      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.4     |
|    ep_rew_mean      | 23.4     |
|    exploration_rate | 0.972    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.5     |
|    ep_rew_mean      | 23.5     |
|    exploration_rate | 0.954    |
| time/               |          |
|    episodes         | 20       |
|    fps              | 3103     |
|    time_elapsed     | 0        |
|    total_timesteps  | 470      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.7     |
|    ep_rew_mean      | 23.7     |
|    exploration_rate | 0.944    |
| time/               |          |
|    episodes         | 24       |
|    fps              | 3440     |
|    time_elapsed     | 0        |
|    total_timesteps  | 569      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.7     |
|    ep_rew_mean      | 23.7     |
|    exploration_rate | 0.944    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 25       |
|    ep_rew_mean      | 25       |
|    exploration_rate | 0.892    |
| time/               |          |
|    episodes         | 44       |
|    fps              | 1228     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1102     |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.0545   |
|    n_updates        | 25       |
----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24.7     |
|    ep_rew_mean      | 24.7     |
|    exploration_rate | 0.884    |
| time/               |          |
|    episodes         | 48       |
|    fps              | 974      |
|    time_elapsed     | 1        |
|    total_timesteps  | 1187     |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.0109   |
|    n_updates        | 46       |
----------------------------------


...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 64.5     |
|    ep_rew_mean      | 64.5     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 1016     |
|    fps              | 402      |
|    time_elapsed     | 246      |
|    total_timesteps  | 99218    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 2.19     |
|    n_updates        | 24554    |
----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 63.4     |
|    ep_rew_mean      | 63.4     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 1020     |
|    fps              | 402      |
|    time_elapsed     | 247      |
|    total_timesteps  | 99408    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.0351   |
|    n_updates        | 24601    |
----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 63.8     |
|    ep_rew_mean      | 63.8     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 1024     |
|    fps              | 402      |
|    time_elapsed     | 247      |
|    total_timesteps  | 99662    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.0571   |
|    n_updates        | 24665    |
----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 66       |
|    ep_rew_mean      | 66       |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 1028     |
|    fps              | 402      |
|    time_elapsed     | 248      |
|    total_timesteps  | 99929    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.0913   |
|    n_updates        | 24732    |
----------------------------------



=== Trained DQN Agent Performance ===
Mean reward: 42.14 ± 3.44

Comparison:
  Random Agent: 21.69 ± 10.88
  Trained DQN:  42.14 ± 3.44
  Improvement:  94.3%


### Stap 4: Visualisatie van de Getrainde Agent

Laten we nu kijken hoe de getrainde agent presteert door een episode te visualiseren.

In [10]:
# Visualize trained agent
def visualize_agent(model, env, n_steps=500):
    """
    Run agent in environment and collect frames for visualization.

    Args:
        model: Trained RL model
        env: Gymnasium environment
        n_steps: Maximum number of steps

    Returns
    -------
        frames, rewards, actions
    """
    frames = []
    rewards_list = []
    actions_list = []

    state, info = env.reset(seed=42)
    frames.append(env.render())

    for _ in range(n_steps):
        # Get action from trained policy (deterministic)
        action, _states = model.predict(state, deterministic=True)
        actions_list.append(int(action))

        # Take action in environment
        state, reward, done, truncated, info = env.step(action)
        rewards_list.append(reward)
        frames.append(env.render())

        if done or truncated:
            break

    return frames, rewards_list, actions_list


# Create environment with rendering
env_render = gym.make("CartPole-v1", render_mode="rgb_array")
frames, rewards_list, actions_list = visualize_agent(model, env_render, n_steps=500)
env_render.close()

print(f"\nEpisode lasted {len(rewards_list)} steps")
print(f"Total reward: {sum(rewards_list):.0f}")
print(f"Action distribution: LEFT={actions_list.count(0)}, RIGHT={actions_list.count(1)}")

# If performance is poor, remind user to retrain
if sum(rewards_list) < 100:
    print("\n⚠️  Poor performance detected!")
    print("Make sure you've run the training cell (cell 10) with the updated hyperparameters.")
    print("The model should achieve close to 500 steps after proper training.")



Episode lasted 39 steps
Total reward: 39
Action distribution: LEFT=17, RIGHT=22

⚠️  Poor performance detected!
Make sure you've run the training cell (cell 10) with the updated hyperparameters.
The model should achieve close to 500 steps after proper training.


Authorization required, but no authorization protocol specified



### Analyse: Hoe Werkt DQN?

DQN leert een **Q-function** die voor elke state-action combinatie schat wat de verwachte return is. Laten we dit visualiseren.

In [ ]:
# Analyze Q-values for different states
def analyze_q_values(model, env, n_samples=100):
    """
    Sample random states and analyze Q-values.

    Args:
        model: Trained DQN model
        env: Gymnasium environment
        n_samples: Number of states to sample

    Returns:
        states, q_values, actions
    """
    states = []
    q_values_left = []
    q_values_right = []
    chosen_actions = []

    for _ in range(n_samples):
        state, _ = env.reset()
        states.append(state)

        # Get Q-values for both actions
        with torch.no_grad():
            q_values = model.q_net(torch.FloatTensor(state).unsqueeze(0))
            q_values_left.append(q_values[0, 0].item())
            q_values_right.append(q_values[0, 1].item())
            chosen_actions.append(torch.argmax(q_values).item())

    return np.array(states), q_values_left, q_values_right, chosen_actions


# Analyze Q-values
states, q_left, q_right, actions = analyze_q_values(model, env, n_samples=200)

print("\n=== Q-Value Analysis ===")
print(f"Average Q-value for LEFT: {np.mean(q_left):.2f}")
print(f"Average Q-value for RIGHT: {np.mean(q_right):.2f}")
print(f"Q-value range: [{min(q_left + q_right):.2f}, {max(q_left + q_right):.2f}]")

# Q-values vs Cart Position
df_pos = pd.DataFrame(
    {
        "Cart Position": np.concatenate([states[:, 0], states[:, 0]]),
        "Q-value": q_left + q_right,
        "Action": ["LEFT"] * len(q_left) + ["RIGHT"] * len(q_right),
    }
)
px.scatter(
    df_pos,
    x="Cart Position",
    y="Q-value",
    color="Action",
    title="Q-values vs Cart Position",
    color_discrete_map={"LEFT": "blue", "RIGHT": "red"},
).show()

# Q-values vs Pole Angle
df_angle = pd.DataFrame(
    {
        "Pole Angle (radians)": np.concatenate([states[:, 2], states[:, 2]]),
        "Q-value": q_left + q_right,
        "Action": ["LEFT"] * len(q_left) + ["RIGHT"] * len(q_right),
    }
)
px.scatter(
    df_angle,
    x="Pole Angle (radians)",
    y="Q-value",
    color="Action",
    title="Q-values vs Pole Angle",
    color_discrete_map={"LEFT": "blue", "RIGHT": "red"},
).show()

# Action Preference vs Pole Angle
q_diff = np.array(q_right) - np.array(q_left)
df_diff = pd.DataFrame(
    {
        "Pole Angle (radians)": states[:, 2],
        "Q(RIGHT) - Q(LEFT)": q_diff,
        "Chosen Action": ["LEFT" if a == 0 else "RIGHT" for a in actions],
    }
)
px.scatter(
    df_diff,
    x="Pole Angle (radians)",
    y="Q(RIGHT) - Q(LEFT)",
    color="Chosen Action",
    title="Action Preference vs Pole Angle",
    color_discrete_map={"LEFT": "blue", "RIGHT": "red"},
).show()

# Action Distribution
action_counts = pd.DataFrame(
    {"Action": ["LEFT", "RIGHT"], "Frequency": [actions.count(0), actions.count(1)]}
)
px.bar(
    action_counts,
    x="Action",
    y="Frequency",
    title="Action Distribution in Sampled States",
    color="Action",
    color_discrete_map={"LEFT": "blue", "RIGHT": "red"},
).show()


## Vergelijking met Andere RL Algoritmes

Laten we nu ook **PPO (Proximal Policy Optimization)** trainen - een policy-based methode - om te zien hoe verschillende RL algoritmes presteren op hetzelfde probleem.

In [ ]:
# Import PPO
from stable_baselines3 import PPO

# Create fresh environment
env_ppo = gym.make("CartPole-v1")

# Create PPO model
# PPO learns a policy π(a|s) directly, not Q-values
model_ppo = PPO(
    "MlpPolicy",
    env_ppo,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    verbose=0,
)

print("=== Training PPO ===")
print("Training for 50,000 timesteps...")

# Train PPO
model_ppo.learn(total_timesteps=50000, progress_bar=True)

print("\n✓ PPO Training completed!")
print("\nEvaluating PPO model...")

# Evaluate PPO (wrap env with Monitor to avoid warning)
eval_env_ppo = Monitor(gym.make("CartPole-v1"))
mean_reward_ppo, std_reward_ppo = evaluate_policy(
    model_ppo, eval_env_ppo, n_eval_episodes=100, deterministic=True
)
eval_env_ppo.close()

print(f"\n=== Algorithm Comparison ===")
print(f"Random Agent: {np.mean(random_rewards):.2f} ± {np.std(random_rewards):.2f}")
print(f"DQN Agent:    {mean_reward:.2f} ± {std_reward:.2f}")
print(f"PPO Agent:    {mean_reward_ppo:.2f} ± {std_reward_ppo:.2f}")

# Visualize comparison
df_comparison = pd.DataFrame(
    {
        "Algorithm": ["Random", "DQN", "PPO"],
        "Mean Reward": [np.mean(random_rewards), mean_reward, mean_reward_ppo],
        "Std": [np.std(random_rewards), std_reward, std_reward_ppo],
    }
)
fig = px.bar(
    df_comparison,
    x="Algorithm",
    y="Mean Reward",
    error_y="Std",
    title="Algorithm Performance Comparison on CartPole-v1",
    color="Algorithm",
    color_discrete_map={"Random": "gray", "DQN": "blue", "PPO": "green"},
    text=[f"{m:.1f}±{s:.1f}" for m, s in zip(df_comparison["Mean Reward"], df_comparison["Std"])],
)
fig.add_hline(y=500, line_dash="dash", line_color="red", annotation_text="Maximum possible (500)")
fig.update_layout(yaxis_range=[0, 550])
fig.show()

## Meer Complexe RL Omgeving: LunarLander

Laten we nu een complexer probleem bekijken: **LunarLander-v2**. Hier moet een maanlander veilig landen op een landingsplatform.

![LunarLander](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/lunarLander.gif)

*Bron: [Hugging Face Deep RL Course](https://huggingface.co/learn/deep-rl-course)*

### Het Probleem
- **State**: 8 continue waarden (positie, snelheid, hoek, hoeksnelheid, been-contact)
- **Actions**: 4 discrete acties (niets, linker motor, hoofd motor, rechter motor)
- **Rewards**: 
  - +100 tot +140 voor succesvolle landing
  - -100 voor crash
  - Kleine negatieve rewards voor brandstofverbruik
  - Positieve rewards voor dichter bij landingszone
- **Doel**: Land veilig met minimaal brandstofverbruik


In [ ]:
# Create LunarLander environment
env_lunar = gym.make("LunarLander-v2")

# Explore the environment
state, info = env_lunar.reset(seed=42)

print("=== LunarLander-v2 Environment ===")
print(f"State space: {env_lunar.observation_space}")
print(f"Action space: {env_lunar.action_space}")
print(f"\nInitial state shape: {state.shape}")
print(f"State: {state}")
print("\nState components:")
print("  [0] X position")
print("  [1] Y position")
print("  [2] X velocity")
print("  [3] Y velocity")
print("  [4] Angle")
print("  [5] Angular velocity")
print("  [6] Left leg contact (0=no, 1=yes)")
print("  [7] Right leg contact (0=no, 1=yes)")
print("\nActions:")
print("  0: Do nothing")
print("  1: Fire left engine")
print("  2: Fire main engine")
print("  3: Fire right engine")

# Test random agent on LunarLander
print("\n=== Testing Random Agent ===")
random_rewards_lunar = evaluate_random_agent(env_lunar, n_episodes=20, seed=42)
print(f"Random Agent: {np.mean(random_rewards_lunar):.2f} ± {np.std(random_rewards_lunar):.2f}")
print("(Note: Negative rewards mean crashes!)")


### Training PPO op LunarLander

Voor dit complexere probleem gebruiken we PPO, dat goed presteert op continue control taken.

In [ ]:
# Train PPO on LunarLander
print("=== Training PPO on LunarLander ===")
print("This will take a few minutes...")

model_lunar = PPO(
    "MlpPolicy",
    env_lunar,
    learning_rate=3e-4,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    clip_range=0.2,
    verbose=0,
)

# Train for longer since this is more complex
model_lunar.learn(total_timesteps=500000, progress_bar=True)

print("\n✓ Training completed!")

# Evaluate (wrap env with Monitor to avoid warning)
eval_env_lunar = Monitor(gym.make("LunarLander-v2"))
mean_reward_lunar, std_reward_lunar = evaluate_policy(
    model_lunar, eval_env_lunar, n_eval_episodes=50, deterministic=True
)
eval_env_lunar.close()

print(f"\n=== LunarLander Results ===")
print(f"Random Agent: {np.mean(random_rewards_lunar):.2f} ± {np.std(random_rewards_lunar):.2f}")
print(f"Trained PPO:  {mean_reward_lunar:.2f} ± {std_reward_lunar:.2f}")
print(f"\nNote: Score > 200 is considered solved!")
status = "SOLVED ✓" if mean_reward_lunar > 200 else "Needs more training"
print(f"Status: {status}")

# Visualize performance
df_lunar = pd.DataFrame(
    {
        "Algorithm": ["Random", "PPO"],
        "Mean Reward": [np.mean(random_rewards_lunar), mean_reward_lunar],
        "Std": [np.std(random_rewards_lunar), std_reward_lunar],
    }
)
fig = px.bar(
    df_lunar,
    x="Algorithm",
    y="Mean Reward",
    error_y="Std",
    title="LunarLander-v2 Performance",
    color="Algorithm",
    color_discrete_map={"Random": "gray", "PPO": "green"},
)
fig.add_hline(y=200, line_dash="dash", line_color="red", annotation_text="Solved threshold (200)")
fig.add_hline(y=0, line_color="black")
fig.show()

### Visualisatie van Getrainde LunarLander Agent

In [ ]:
# Visualize trained LunarLander agent
env_lunar_render = gym.make("LunarLander-v2", render_mode="rgb_array")
frames_lunar, rewards_lunar, actions_lunar = visualize_agent(
    model_lunar, env_lunar_render, n_steps=500
)
env_lunar_render.close()

print(f"\n=== Episode Analysis ===")
print(f"Episode length: {len(rewards_lunar)} steps")
print(f"Total reward: {sum(rewards_lunar):.1f}")
print(
    f"Final outcome: {'SUCCESS ✓' if sum(rewards_lunar) > 200 else 'CRASH' if sum(rewards_lunar) < 0 else 'PARTIAL'}"
)
print(f"\nAction usage:")
action_names = ["Do nothing", "Left engine", "Main engine", "Right engine"]
for action_id, action_name in enumerate(action_names):
    count = actions_lunar.count(action_id)
    percentage = (count / len(actions_lunar)) * 100
    print(f"  {action_name}: {count} times ({percentage:.1f}%)")

# Show action distribution
df_actions = pd.DataFrame(
    {"Action": action_names, "Count": [actions_lunar.count(i) for i in range(4)]}
)
px.bar(
    df_actions,
    x="Action",
    y="Count",
    title=f"LunarLander Action Distribution (Total Reward: {sum(rewards_lunar):.1f})",
).show()
